# Embeddings

In [1]:
import os, sys

import numpy as np
import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from log_files import LogData
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction, TfidfFeatureExtraction

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Read csv files and load as df

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
predictions_df = log_files.read_data(notebook_dir, log_file_path, predictions)
predictions_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/scratch-predictions/misc_experiments/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/scratch-predictions/misc_experiments/../data/prediction_logs/batch_1-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/scratch-predictions/misc_experiments/../data/prediction_logs/batch_1-prediction/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/scratch-predictions/misc_experiments/../data/prediction_logs/batch_2-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/scratch-predictions/misc_experiments/../data/prediction_logs/batch_2-prediction/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/re

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,1,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [4]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/scratch-predictions/misc_experiments/../data/observation_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/scratch-predictions/misc_experiments/../data/observation_logs/batch_1-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/scratch-predictions/misc_experiments/../data/observation_logs/batch_1-observation/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/scratch-predictions/misc_experiments/../data/observation_logs/batch_2-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/scratch-predictions/misc_experiments/../data/observation_logs/batch_2-observation/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Devel

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"A young investor predicts on 2025-03-15, the S&P 500 index may rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [5]:
col_name = 'Base Sentence'
predictions = DataProcessing.df_to_list(predictions_df, col_name)
observations = DataProcessing.df_to_list(observations_df, col_name)
len(predictions), len(observations)

(430, 1435)

## Extract Spacy embeddings

In [6]:
disable_components = [""]
pred_spacy_fe = SpacyFeatureExtraction(predictions_df, "Base Sentence")
pred_sentence_features = pred_spacy_fe.sentence_feature_extraction()

obser_spacy_fe = SpacyFeatureExtraction(observations_df, "Base Sentence")
obser_sentence_features = obser_spacy_fe.sentence_feature_extraction()

100%|██████████| 3/3 [00:00<00:00, 246.38it/s]


In [7]:
spacy_pred_sent_embedding = pred_sentence_features[0]
spacy_obser_sent_embedding = obser_sentence_features[0]
# spacy_pred_sent_embedding.shape

In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

sentences = [] 
sentences.append(predictions[0])
sentences.append(observations[0])
print(sentences)

embeddings = model.encode(sentences)
print(embeddings.shape)
# => (3, 384)


# spacy_pred_sent_embedding.shape

similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

sent_transformer_pred_sent_embedding = model.encode(predictions[0])
sent_transformer_obser_sent_embedding = model.encode(observations[0])
similarities = model.similarity(sent_transformer_pred_sent_embedding, sent_transformer_obser_sent_embedding)
print(similarities)


['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.', 'The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(2, 384)
tensor([[1.0000, 0.3989],
        [0.3989, 1.0000]])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([[0.3989]])


In [9]:
import torch
from transformers import AutoTokenizer, AutoModel

from transformers import BertTokenizer, BertModel
def get_bert_embeddings(sentence: str):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained("bert-base-uncased")
    # text = "Replace me by any text you'd like."
    encoded_input = tokenizer(sentence, return_tensors='pt')
    # Get hidden states from BERT
    with torch.no_grad():
        output = model(**encoded_input)

    # Extract embeddings for [CLS] token
    sentence_embedding = output.last_hidden_state[:, 0, :].squeeze()

    return sentence_embedding

In [10]:
bert_pred_sent_embedding = get_bert_embeddings(predictions[0])
bert_obser_sent_embedding = get_bert_embeddings(observations[0])

In [11]:
pred_tfidf_fe = TfidfFeatureExtraction(predictions_df, 'Base Sentence')
obser_tfidf_fe = TfidfFeatureExtraction(observations_df, 'Base Sentence')

pred_tfidf_df = pred_tfidf_fe.feature_scores(max_features=300)
obser_tfidf_df = obser_tfidf_fe.feature_scores(max_features=300)

In [12]:
pred_tfidf_df[:1]

,Base Sentence,Sentence Label,01,03,06,08,09,10,12,15,20,2024,2025,2026,2027,2028,2029,21,22,500,according,accuweather,activity,administration,advisor,agency,alex,alphabet,amazon,america,american,among,analyst,and,angeles,apple,areas,association,at,atlanta,atmospheric,august,average,bank,barcelona,based,boston,breaches,brown,budget,bureau,by,cafeterias,cash,cdc,celtics,centers,chances,channel,chase,chen,chicago,chronic,citigroup,climate,clinics,coach,coca,cola,college,community,companies,company,conducted,confidence,consumption,control,could,count,customer,cybersecurity,dallas,data,david,davis,decrease,defense,denver,development,diabetes,disease,dr,economic,economist,electric,emerging,emily,energy,engagement,england,environmental,envisions,espn,european,executive,expected,expenses,expert,exxonmobil,fall,fargo,fc,federal,finance,financial,fitness,flow,for,forecasts,francisco,from,gamble,game,goal,golden,goldman,google,gross,growth,harvard,has,have,health,healthcare,heart,her,high,hospitals,houston,humidity,illnesses,in,income,increase,inflation,initiatives,institutions,intake,interest,international,investments,james,january,john,johnson,jpmorgan,kevin,kim,lakers,late,lee,level,levels,likely,lisa,local,los,major,market,marketing,markets,may,meteorologist,miami,michael,microsoft,might,miscellaneous,morgan,mr,ms,national,net,new,nguyen,noaa,november,number,nutritional,obesity,oceanic,october,of,on,operating,organization,orleans,patel,patriots,per,percentage,person,physical,point,points,policy,potentially,precipitation,predicts,prevalence,prevention,price,prices,probability,procter,professor,profit,protection,public,q1,q2,q3,q4,rachel,rate,rates,remain,renewable,report,reporter,research,reserve,revenue,rise,robert,rodriguez,rural,sachs,salary,sales,same,san,sarah,school,schools,scoring,sector,senator,senior,september,service,should,smith,some,sophia,speculates,speed,spending,sport,sports,stable,stanley,state,states,stay,stock,student,study,sugar,taylor,tech,technology,teenagers,temperature,tesla,that,the,thompson,threats,to,tom,top,touchdown,turnout,unemployment,united,university,urban,us,vehicles,visa,voter,weather,wells,white,will,win,wind,world,would,yankees,york
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.229623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.330718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.106179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.412087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.223564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.227143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.103877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.357931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.155066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.322028,0.0,0.0,0.0,0.0,0.204972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.138118,0.082033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
tfidf_pred_embedding = pred_tfidf_df.iloc[:1 , 2:].to_numpy()

In [14]:
obser_tfidf_df[:1]

,Base Sentence,Sentence Label,01,02,03,05,06,07,08,09,10,11,12,15,20,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,21,22,25,according,accuracy,activity,administration,advisor,air,alex,amazon,amelia,america,american,among,analyst,and,angeles,apple,areas,association,at,atmospheric,aug,august,average,bank,bay,blood,body,boston,brown,budget,bulls,bureau,by,california,cash,cdc,center,centers,changed,chase,chen,chicago,children,citigroup,city,clinics,cloud,coach,college,community,company,conducted,control,count,countries,coverage,customer,david,davis,decreased,denver,developing,development,diabetes,disease,dr,economic,economist,elementary,emily,energy,england,envisioned,envisions,european,executive,expectancy,expenses,expert,fall,fallen,fargo,february,federal,fell,financial,flow,for,fourth,francisco,free,from,funding,goal,golden,goldman,google,green,gross,growth,had,hall,harvard,has,have,health,heart,heat,her,high,his,home,hospitals,humidity,in,income,increase,increased,index,inflation,institute,international,investments,james,january,john,johnson,jpmorgan,june,kim,lee,level,levels,life,likely,lisa,local,los,manager,march,market,mass,may,mental,meteorologist,miami,michael,microsoft,miscellaneous,monitored,morgan,mr,ms,national,nations,net,new,nguyen,noted,november,number,obesity,observed,october,of,on,operating,organization,passing,patel,patients,patriots,percentage,person,physical,point,policy,precipitation,predicted,predicts,pressure,prevalence,prevention,price,prices,probability,professor,profit,q1,q2,q3,q4,quarter,rachel,rate,rates,recorded,remain,remained,renewable,reported,representative,research,researcher,reserve,revenue,rise,risen,rodriguez,rose,run,rural,sachs,sales,same,sarah,satisfaction,save,saw,schools,season,seattle,second,senior,september,service,should,significantly,smith,some,sophia,speculated,speculates,speed,sports,stable,staff,stanley,state,states,stay,stayed,stock,student,study,summer,tax,tech,temperature,tesla,that,the,thompson,throw,to,tom,top,unemployment,united,university,urban,vaccination,walmart,warriors,weather,wells,will,wind,world,would,yards,year,york
0,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.163429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.205068,0.0,0.0,0.0,0.0,0.0,0.191307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.255582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.331082,0.0,0.0,0.0,0.0,0.183259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.098609,0.347619,0.0,0.216563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.202059,0.0,0.133212,0.0,0.308043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.269658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.331082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333216,0.121727,0.216822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
tfidf_obser_embedding = obser_tfidf_df.iloc[:1 , 2:].to_numpy()
tfidf_obser_embedding

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.16342928, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.20506823,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.19130674, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine_similarity(prediction_embeddings: np.array, observation_embeddings: np.array):

    # make them (1 × vector_dim) for sklearn
    pred_sent_embedding_reshaped = prediction_embeddings.reshape(1, -1)
    obser_sent_embedding_reshaped = observation_embeddings.reshape(1, -1)

    sim = cosine_similarity(pred_sent_embedding_reshaped, obser_sent_embedding_reshaped)[0, 0]
    
    return sim

In [17]:
spacy_cs_metric = get_cosine_similarity(spacy_pred_sent_embedding, spacy_obser_sent_embedding)
sent_tranformer_cs_metric = get_cosine_similarity(sent_transformer_pred_sent_embedding, sent_transformer_obser_sent_embedding)
bert_cs_metric = get_cosine_similarity(bert_pred_sent_embedding, bert_obser_sent_embedding)
tfidf_cs_metric = get_cosine_similarity(tfidf_pred_embedding, tfidf_obser_embedding)

spacy_cs_metric, sent_tranformer_cs_metric, bert_cs_metric, tfidf_cs_metric

(np.float32(0.8639305),
 np.float32(0.39891112),
 np.float32(0.91758525),
 np.float64(0.0552723886435757))